## Imports

In [1]:
from game import GAME
import torch
import numpy as np

import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

from torch.distributions import Normal

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Instanciando game e exemplificando

In [2]:
env = GAME(draw=True)  
observations = env.reset()

In [3]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if joga_novamente:
        rewards += 1

    if perdeu:
        rewards -= 1.5

    if ganhou:
        rewards += 1.5

    if penalizado:
        rewards -= 1

    return rewards

In [4]:
env.table.draw()

In [5]:
## jogador faz a jogada


env.iniciou_jogada = False
while not env.iniciou_jogada:
    env.table.draw()

obs, information, terminations, rewards = env.step((env.iniciou_jogada_angulo, 
                                                    env.inicou_jogada_intensidade), rewards_function=rewards_function)

print('state', obs)
print('termination', terminations)
print('rewards', rewards)

state (tensor([[0.0137, 0.5000, 1.0000, 0.0000],
        [0.9210, 0.0862, 1.0000, 0.0000],
        [0.7305, 0.0800, 1.0000, 1.0000],
        [0.9072, 0.5875, 1.0000, 0.0000],
        [0.9466, 0.8325, 1.0000, 1.0000],
        [0.9379, 0.7357, 1.0000, 0.0000],
        [0.9423, 0.6031, 1.0000, 1.0000],
        [0.7940, 0.3715, 1.0000, 0.0000],
        [0.4413, 0.7356, 1.0000, 1.0000],
        [0.9355, 0.3417, 1.0000, 0.0000],
        [0.9119, 0.6792, 1.0000, 1.0000],
        [0.8300, 0.7633, 1.0000, 1.0000],
        [0.9534, 0.2136, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]]), tensor([0.4703, 0.5511]))
termination False
rewards 0


In [6]:
# computador faz a jogada
print("Iniciando jogada", env.jogador_atual)



angulo = 10
intensidade = 0.9

env.iniciou_jogada = False
obs, information, terminations, rewards = env.step( (angulo, intensidade),  rewards_function=rewards_function)

print("Recompensa da jogada:", rewards)

Iniciando jogada True
Recompensa da jogada: -1


## Começando treinamento
                           

### ARGS

In [7]:
@dataclass
class Args:
    embed_dim : int = 128
    num_heads : int = 8         
    ff_dim : int = embed_dim * 2
    num_layers : int =  2       
    dropout : float = 0.1         

### Modelo política

A ideia é que tenhamos uma rede neural como política, especificamente um transformers. 

Queremos que ele passe por cada bola e gere uma representação, no fim a representação do nosso estado será o embedding dabola branca.

In [8]:

class Transformers(nn.Module):
    def __init__(self, args : Args):
        super(Transformers, self).__init__()
        
        # Layer de embedding (entra uma bola com (x,y,z, w) e sai um vetor de 128 dimensões que representa a bola)
        self.embedding = nn.Linear(4, args.embed_dim)
        
        # Encoder layers
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer( d_model=args.embed_dim, 
                                       nhead=args.num_heads, 
                                       dim_feedforward=args.ff_dim, 
                                       dropout=args.dropout) for _ in range(args.num_layers)
        ])
        
        # Layer normalization
        self.layer_norm = nn.LayerNorm(args.embed_dim)
        
        #MlP para gerar regressão de angulo e intensidade
        self.mlp = nn.Sequential( 
                nn.Linear(args.embed_dim, args.ff_dim),
                nn.Tanh(),
                nn.Linear(args.ff_dim, 3) # Vamos sair com (sen, cos, intensidade)
        )
        
        self.head_position = nn.Tanh() # Tanh para garantir que o angulo fique entre -1 e 1
        self.head_intensity = nn.Sigmoid() # Sigmoid para garantir que a intensidade fique entre 0 e 1

    def forward(self, x, bola_branca): 
        # adiciona a dimensão da bolinha branca -> batch, (x,y,1,-1)
        b = bola_branca.shape[0]
        t_concat = torch.zeros(b,2, device=bola_branca.device)
        t_concat[:,0] = 1
        t_concat[:,1] = -1
        bola_branca = torch.concat( (bola_branca , t_concat), dim=-1).unsqueeze(1)
        x = torch.concat((bola_branca,x),dim=1)
        
        x = self.embedding(x)        
        
        for layer in self.encoder_layers:
            x = layer(x)
    
        x = self.layer_norm(x)
        
        # bola branca é nosso Value, que ira representar o estado do jogo
        x = x[:,0,:]        
        x = self.mlp(x)
        
        # transforma o x em angulo e intensidade
        position = self.head_position(x[:,:2])
        intensity = self.head_intensity(x[:,2:])
        
        x_sin = position[:,0:1] # seno
        x_cos = position[:,1:2] # cosseno
        
        angles_deg = torch.rad2deg(torch.atan2(x_sin, x_cos)) # converte para graus
        angles_deg = torch.where(angles_deg < 0, angles_deg + 360, angles_deg)
        
        # concatena o resultado
        return torch.concat((angles_deg,intensity),dim=-1) # Retorna o angulo e a intensidade


#### Instanciando e testando modelo

In [9]:
args = Args(
)

model = Transformers(
    args
)


Exemplo de foward:

In [10]:
state, bola_branca = (torch.tensor([
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]
        ]),
                      
 torch.tensor([[0.2000, 0.5000],[0.2000, 0.5000]]))

model(state,bola_branca)

tensor([[350.2479,   0.5267],
        [334.3201,   0.5506]], grad_fn=<CatBackward0>)

### Value model

In [11]:
class TransformerValueModel(nn.Module):
    def __init__(self, args : Args):
        super(TransformerValueModel, self).__init__()
        
        self.embedding = nn.Linear(4, args.embed_dim)
        
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=args.embed_dim,
                nhead=args.num_heads,
                dim_feedforward=args.ff_dim,
                dropout=args.dropout
            ) for _ in range(args.num_layers)
        ])
        
        self.layer_norm = nn.LayerNorm(args.embed_dim)
        
        self.value_head = nn.Sequential(
            nn.Linear(args.embed_dim, args.ff_dim),  # Projeta para dimensão intermediária
            nn.ReLU(),                    # Ativação não-linear
            nn.Linear(args.ff_dim, 1)          # Saída escalar (valor do estado)
        )

    def forward(self, x, bola_branca):
        # Adiciona a dimensão da bolinha branca -> batch, (x,y,1,-1)
        b = bola_branca.shape[0]
        t_concat = torch.zeros(b, 2, device=bola_branca.device)
        t_concat[:, 0] = 1
        t_concat[:, 1] = -1
        bola_branca = torch.concat((bola_branca, t_concat), dim=-1).unsqueeze(1)
        x = torch.concat((bola_branca, x), dim=1)
        
        # Passa pela camada de embedding
        x = self.embedding(x)
        
        # Passa pelas camadas de encoder
        for layer in self.encoder_layers:
            x = layer(x)
        
        # Normalização final
        x = self.layer_norm(x)
        
        # Seleciona a representação do estado do jogo (primeiro token)
        x = x[:, 0, :]
        
        # Passa pela cabeça de valor para prever V(s)
        value = self.value_head(x)
        
        # Garante que a saída está no formato (batch_size,)
        return value  # Remove apenas a última dimensão


In [12]:
args = Args(
    embed_dim=128   
)

model = TransformerValueModel(
    args
)

In [13]:
model(state,bola_branca)

tensor([[0.1074],
        [0.1023]], grad_fn=<AddmmBackward0>)

### PPO

In [14]:
class Agent(nn.Module):
    def __init__(self, action_dim, args):
        super().__init__()
        self.critic = TransformerValueModel(args)  # Modelo para estimar o valor
        self.actor_mean = Transformers(args)  # Modelo para estimar a média das ações
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))  # Desvio padrão fixo

    def get_value(self, state, bola_branca):
        """Obtém o valor estimado pelo modelo crítico."""
        return self.critic(state, bola_branca)

    def get_action_and_value(self, state, bola_branca, action=None):
        """
        Obtém a ação, log_prob, entropia e valor.
        
        Args:
            x: Entrada para o modelo.
            bola_branca: Dados da bola branca.
            action: Ação opcional para avaliar log_prob e entropia.
        
        Returns:
            action: Ação amostrada ou fornecida.
            log_prob: Logaritmo da probabilidade da ação.
            entropy: Entropia da distribuição.
            value: Valor estimado pelo crítico.
        """
        # O ator retorna ângulo e intensidade
        mean = self.actor_mean(state, bola_branca)  # Predição da média das ações
        log_std = self.actor_log_std.expand_as(mean)  # Ajusta o desvio padrão ao formato correto
        std = torch.exp(log_std)  # Calcula o desvio padrão

        # Distribuição Normal para ações contínuas
        dist = Normal(mean, std)

        if action is None:
            action = dist.sample()  # Amostra uma ação da distribuição

        log_prob = dist.log_prob(action).sum(axis=-1)  # Log probabilidade
        entropy = dist.entropy().sum(axis=-1)  # Entropia

        value = self.critic(state, bola_branca)  # Valor estimado pelo crítico

        return action, log_prob, entropy, value

In [15]:
action_dim = 2

In [16]:
agent = Agent(action_dim, args)

In [17]:
action, log_prob, entropy, value = agent.get_action_and_value(state, bola_branca)

print('action', action)
print('log_prob', log_prob)
print('entropy', entropy)
print('value', value)

action tensor([[ 97.5531,   1.1060],
        [162.2326,   1.0374]])
log_prob tensor([-2.2460, -2.1543], grad_fn=<SumBackward1>)
entropy tensor([2.8379, 2.8379], grad_fn=<SumBackward1>)
value tensor([[0.1254],
        [0.0573]], grad_fn=<AddmmBackward0>)


Aqui vemos que as ações não estão no range esperado, isso porque estamos passando uma distribuiçao normal 

In [18]:
class PPO:
    def __init__(self, policy_model, value_model ,lr=1e-3, gamma=0.99, eps_clip=0.2, entropy_coef=0.01):
        self.policy = policy_model
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.entropy_coef = entropy_coef
        
        self.value_model = value_model
        self.value_optimizer = optim.Adam(self.value_model.parameters(), lr=lr)
        
    def compute_advantage(self, rewards, values, masks, gamma):
        returns = []
        R = 0
        for reward, mask in zip(reversed(rewards), reversed(masks)):
            R = reward + gamma * R * mask
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)
        advantage = returns - values
        return advantage.detach(), returns
    
    def update(self, states, bola_branca, actions, log_probs_old, rewards, dones):
        states_flat = states.view(states.size(0), -1)
        batch_size = states.size(0)

        # Calculando valores e vantagens
        values = self.value_model(states_flat, bola_branca).squeeze(-1)
        advantages, returns = self.compute_advantage(rewards, values, 1 - dones, self.gamma)
        
        # Atualizar política
        for _ in range(4):  # Número de atualizações
            outputs = self.policy(states, bola_branca)
            dist = Normal(outputs, torch.ones_like(outputs))  # Normal para distribuição
            log_probs = dist.log_prob(actions).sum(-1)
            entropy = dist.entropy().sum(-1)
            
            # Clipped Surrogate Objective
            ratio = torch.exp(log_probs - log_probs_old)
            surrogate1 = ratio * advantages
            surrogate2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            policy_loss = -torch.min(surrogate1, surrogate2).mean()
            entropy_loss = -self.entropy_coef * entropy.mean()
            
            self.optimizer.zero_grad()
            (policy_loss + entropy_loss).backward()
            self.optimizer.step()

        # Atualizar função de valor
        for _ in range(4):  # Número de atualizações
            values = self.value_model(torch.cat((states_flat, bola_branca), dim=1)).squeeze(-1)
            value_loss = nn.MSELoss()(values, returns)
            
            self.value_optimizer.zero_grad()
            value_loss.backward()
            self.value_optimizer.step()

In [19]:
policy = Transformers(args)
value = TransformerValueModel(args)

In [20]:
ppo = PPO(policy, value)

In [ ]:
import utils.config as cfg

## iniciar o ambiente com a configuração do jogo com apenas uma bola na mesa. 
# - A bola branca na posição (0, 0)
# - se não bater na bola, acabou a jogada

cfg.raio_buraco = 13
env.reset()
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]
numero_bola = env.table.bolas[0].numero
env.numero_bolas = [[ numero_bola ], [ numero_bola ]]

#env.table.draw()

In [20]:
state_bolas, state_bola_branca = env.get_observations()
# shuffle the balls
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]
state_bolas , state_bola_branca

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.2000, 0.5000]))